In [ ]:
import os
import numpy as np
#os.environ['AUTOGRAPH_VERBOSITY'] = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["PYTHONHASHSEED"] = "0"
#os.environ['TF_DETERMINISTIC_OPS'] = '1' 
import tensorflow as tf
import random

In [ ]:
seed = 100
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [ ]:
print(tf.__version__)

In [ ]:
dataset_name='your path'
dataset_folder='your path'
preflix_folder='your path'

In [ ]:
import platform
if platform.system().lower() == 'windows':
    import pickle5 as pickle
else:
    import pickle

items_num=38048
users_num=31668

batch_size=2048
epoch_num=1000
hidden_size=64
keep_rate=0.9
layers_num=1
neg_layers_num=1
loss_ratio=0.0001
group_num=3
eigen_dim=64


neg_num=1

In [ ]:
print(items_num)

In [ ]:
if platform.system().lower() == 'windows':
    print("windows")
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_train_data_index_from_0.pkl', 'rb') as f:
        train_data=pickle.load(f)
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_test_data_index_from_0.pkl', 'rb') as f:
        test_data=pickle.load(f)
    with open('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_val_data_index_from_0.pkl', 'rb') as f:
        val_data=pickle.load(f)
elif platform.system().lower() == 'linux':
    print("linux")
    with open('/root/autodl-fs/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_train_data_index_from_0.pkl', 'rb') as f:
        train_data=pickle.load(f)
    with open('/root/autodl-fs/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_test_data_index_from_0.pkl', 'rb') as f:
        test_data=pickle.load(f)
    with open('/root/autodl-fs/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_gcn_val_data_index_from_0.pkl', 'rb') as f:
        val_data=pickle.load(f)

In [ ]:
print(np.shape(train_data['pos_item']),np.shape(test_data['pos_item']),np.shape(val_data['pos_item']))

In [ ]:
users_num_max=max(max(train_data['users']),max(test_data['users']),max(val_data['users']))
users_num_min=min(min(train_data['users']),min(test_data['users']),min(val_data['users']))
items_num_max=max(max(train_data['pos_item']),max(test_data['pos_item']),max(val_data['pos_item']))
items_num_min=min(min(train_data['pos_item']),min(test_data['pos_item']),min(val_data['pos_item']))
print('users range:[',users_num_min,',',users_num_max,']')
print('items range:[',items_num_min,',',items_num_max,']')

In [ ]:
import scipy.sparse as sp
if platform.system().lower() == 'windows':
    print("windows")
    pre_adj_mat = sp.load_npz('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_s_pre_adj_mat_index_from_0.npz')#D^-1/2*A*D^-1/2
elif platform.system().lower() == 'linux':
    print("linux")
    pre_adj_mat = sp.load_npz('/root/autodl-fs/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_s_pre_adj_mat_index_from_0.npz')#D^-1/2*A*D^-1/2
print('already load adj matrix', pre_adj_mat.shape)

In [ ]:
norm_adj=pre_adj_mat

In [ ]:
try: 
    if platform.system().lower() == 'windows':
        tmp=np.load('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_eig_val_vec.npz')#D^-1/2*A*D^-1/2
        vals_np=tmp['arr_0']
        vecs_np=tmp['arr_1']
    elif platform.system().lower() == 'linux':
        tmp=np.load('/root/autodl-fs/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_eig_val_vec.npz')#D^-1/2*A*D^-1/2
        vals_np=tmp['arr_0']
        vecs_np=tmp['arr_1']
    print(platform.system().lower(),'already load eigs', vals_np.shape, vecs_np.shape)
except :
    vals_np, vecs_np = sp.linalg.eigsh(norm_adj,128)
    if platform.system().lower() == 'windows':
        np.savez('E:/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_eig_val_vec.npz',vals_np,vecs_np)#D^-1/2*A*D^-1/2
    elif platform.system().lower() == 'linux':
        np.savez('/root/autodl-fs/workspace/datasets/'+dataset_folder+'/'+dataset_name+'/'+preflix_folder+'/'+dataset_name+'_eig_val_vec.npz',vals_np,vecs_np)#D^-1/2*A*D^-1/2
    print(platform.system().lower(),'already save eig matrix', vals_np.shape, vecs_np.shape)

In [ ]:
structure_fea=vecs_np[:,-(eigen_dim+1):-1]

In [ ]:
class QueryLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_dim):
        super(QueryLayer, self).__init__()
        self.embedding_dim = embedding_dim
        self.dense1 = tf.keras.layers.Dense(2*self.embedding_dim)
        self.tanh = tf.keras.layers.Activation('tanh')
        self.dense2 = tf.keras.layers.Dense(group_num)
        self.softmax = tf.keras.layers.Activation('softmax')

    def call(self, inputs,temperature=0.01):
        x = self.dense1(inputs)
        x = self.tanh(x)
        x = self.dense2(x)
        x = self.softmax(x/temperature)
        return x
        #max_index = tf.argmax(x,axis=1)
        #one_hot_vector = tf.one_hot(max_index, depth=x.shape[1])
        #return one_hot_vector

In [ ]:
class lightgcn_layer(tf.keras.layers.Layer):
    
    def reshapes(self,embed1):
        return tf.reshape(embed1, [-1,hidden_size])
    
    def sign(self,x):
        return tf.nn.tanh(100*x)
    
    
    def __init__(self,norm_adj,structure_fea,hidden_size=64, dropout=0., embed_reg=1e-6):
        
        super(lightgcn_layer, self).__init__()


        #self.vecs_np,self.p1,self.p2=vecs_np,p1,p2
        self.norm_adj=norm_adj
        self.structure_fea=structure_fea
        


        self.dropout = tf.keras.layers.Dropout(dropout)
        self.query=QueryLayer(hidden_size)
        self.query_=QueryLayer(hidden_size)
        
        #self.k=tf.Variable(2*tf.ones((group_num,1),dtype=tf.float32),trainable=True)
        self.k = tf.Variable(tf.random.uniform((group_num, 1), minval=-5, maxval=5, dtype=tf.float32, seed=2024), trainable=True)
        #self.k = tf.Variable(tf.random.uniform((group_num, 1), minval=-5, maxval=5, dtype=tf.float32), trainable=True)
        self.k_={}
        for i in range(layers_num-1):
            self.k_[str(i)]=tf.Variable(tf.ones((group_num,1),dtype=tf.float32),trainable=True)
        self.query_={}
        for i in range(layers_num-1):
            self.query_[str(i)]=QueryLayer(hidden_size)
        self.w1 = tf.keras.layers.Dense(hidden_size,use_bias=False)
        self.w2 = tf.keras.layers.Dense(32,use_bias=False)
        
        #self.k=tf.Variable([1.],trainable=True)

    
    def perjinsi(self,k,query,user_embedding,item_embedding,training):
        #ego_embeddings = tf.concat(
        #    [user_embedding,item_embedding], axis=0
        #)
        #side_embeddings = tf.sparse.sparse_dense_matmul(self.norm_adj, ego_embeddings)
        #side_ua_embeddings, side_ia_embeddings = tf.split(
        #    side_embeddings, [users_num,items_num], 0
        #)
        #fea_u=tf.concat([user_embedding,side_ua_embeddings,user_embedding+side_ua_embeddings,tf.multiply(user_embedding,side_ua_embeddings)],axis=-1)
        #fea_i=tf.concat([item_embedding,side_ia_embeddings,item_embedding+side_ia_embeddings,tf.multiply(item_embedding,side_ia_embeddings)],axis=-1)
        fea_u, fea_i = tf.split(
            self.structure_fea, [users_num,items_num], 0
        )
        k_aft_q_u=query(fea_u)+query(-fea_u)
        k_aft_q_i=query(fea_i)+query(-fea_i)
        k_aft_q=tf.concat([tf.matmul(k_aft_q_u,k),tf.matmul(k_aft_q_i,k)], axis=0)
        max_0_k = tf.nn.relu(k_aft_q)
        min_0_k =  -tf.nn.relu(-k_aft_q)
        max_0_neg_sign_k = tf.maximum(0.0, -self.sign(k_aft_q+1e-8))
        '''pool_weights_part1=tf.stack([tf.ones_like(self.max_0_k),self.max_0_k,self.max_0_k*(self.max_0_k-1.)/2,
                              self.max_0_k*(self.max_0_k-1.)*(self.max_0_k-2.)/6,
                              self.max_0_k*(self.max_0_k-1.)*(self.max_0_k-2.)*(self.max_0_k-3.)/24,
                              self.max_0_k*(self.max_0_k-1.)*(self.max_0_k-2.)*(self.max_0_k-3.)*(self.max_0_k-4.)/120],axis=1)#[u+i,layer_num+1,1]
        
        pool_weights_part2=tf.stack([tf.pow(3.,self.min_0_k)*tf.ones_like(self.min_0_k),-tf.pow(3.,self.min_0_k)*self.min_0_k/3,tf.pow(3.,self.min_0_k)*self.min_0_k*(self.min_0_k-1.)/18,
                              -tf.pow(3.,self.min_0_k)*self.min_0_k*(self.min_0_k-1.)*(self.min_0_k-2.)/126,
                              tf.pow(3.,self.min_0_k)*self.min_0_k*(self.min_0_k-1.)*(self.min_0_k-2.)*(self.min_0_k-3.)/1944,
                              -tf.pow(3.,self.min_0_k)*self.min_0_k*(self.min_0_k-1.)*(self.min_0_k-2.)*(self.min_0_k-3.)*(self.min_0_k-4.)/29160],axis=1)#[u+i,layer_num+1,1]'''
        
        pool_weights=tf.stack([tf.pow(0.5,max_0_k)*tf.pow(0.5,min_0_k)*tf.pow(3.,min_0_k)*tf.ones_like(min_0_k),
                               tf.pow(0.5,max_0_k)*tf.pow(0.5,min_0_k)*tf.pow(3.,min_0_k)*(3*max_0_k-min_0_k)/3,
                               tf.pow(0.5,max_0_k)*tf.pow(0.5,min_0_k)*tf.pow(3.,min_0_k)*(9*max_0_k*max_0_k+(-6*min_0_k-9)*max_0_k+min_0_k*min_0_k-min_0_k)/18,
                               tf.pow(0.5,max_0_k)*tf.pow(0.5,min_0_k)*tf.pow(3.,min_0_k)*(27*max_0_k*max_0_k*max_0_k+(-27*min_0_k-81)*max_0_k*max_0_k+\
                                (9*min_0_k*min_0_k+18*min_0_k+54)*max_0_k-min_0_k*min_0_k*min_0_k+3*min_0_k*min_0_k-2*min_0_k)/162,
                              tf.pow(0.5,max_0_k)*tf.pow(0.5,min_0_k)*tf.pow(3.,min_0_k)*(81*tf.pow(max_0_k,4)+(-108*min_0_k-486)*tf.pow(max_0_k,3)+(54*tf.pow(min_0_k,2)+270*min_0_k+891)*tf.pow(max_0_k,2)+\
                                (-12*tf.pow(min_0_k,3)-18*tf.pow(min_0_k,2)-186*min_0_k-486)*max_0_k+tf.pow(min_0_k,4)-6*tf.pow(min_0_k,3)+11*tf.pow(min_0_k,2)-6*min_0_k)],axis=1)
        all_embeddings = tf.concat([self.user_embedding,self.item_embedding], axis=0)
        ego_embeddings = tf.concat(
            [self.user_embedding,self.item_embedding], axis=0
        )
        ori_embeddings=ego_embeddings
        
        all_embeddings = [ego_embeddings]
        for l in range(0, 4):
            
            ego_embeddings = tf.sparse.sparse_dense_matmul(self.norm_adj, ego_embeddings)
            if training:
                ego_embeddings=self.dropout(ego_embeddings)
            
            all_embeddings += [ego_embeddings]
        all_embeddings = tf.stack(all_embeddings, 1)
        all_embeddings=tf.multiply(all_embeddings,pool_weights)
        #tf.print(all_embeddings.shape)
        all_embeddings = tf.reduce_sum(input_tensor=all_embeddings, axis=1, keepdims=False)
        part_1=tf.multiply(max_0_neg_sign_k,ego_embeddings)
        all_embeddings=all_embeddings+part_1
        g1_ua_embeddings, g1_ia_embeddings = tf.split(
            all_embeddings, [users_num,items_num], 0
        )
        return g1_ua_embeddings,g1_ia_embeddings
    
    def call(self,item_embedding_pool,user_embedding_pool,training=False):
        #vecs_np,p1,p2=tf.constant(vecs_np),tf.constant(p1),tf.constant(p2)
        self.item_embedding=item_embedding_pool(tf.range(items_num))
        self.user_embedding=user_embedding_pool(tf.range(users_num))
        g1_ua_embeddings,g1_ia_embeddings=self.perjinsi(self.k,self.query,self.user_embedding,self.item_embedding,training)
        all_embeddings =[tf.concat([g1_ua_embeddings,g1_ia_embeddings],axis=0)]
        for i in range(layers_num-1):
            g1_ua_embeddings,g1_ia_embeddings=self.perjinsi(self.k_[str(i)],self.query_[str(i)],g1_ua_embeddings,g1_ia_embeddings,training)
            all_embeddings += [tf.concat([g1_ua_embeddings,g1_ia_embeddings],axis=0)]
        all_embeddings = tf.stack(all_embeddings, 1)
        all_embeddings = tf.math.reduce_mean(all_embeddings, 1)
        #g1_ua_embeddings, g1_ia_embeddings = tf.split(
        #    all_embeddings, [users_num,items_num], 0
        #)
        
        return tf.reshape(g1_ua_embeddings,[-1,hidden_size]),tf.reshape(g1_ia_embeddings,[-1,hidden_size])

In [ ]:
class lightgcn(tf.keras.models.Model):

    def _convert_sp_mat_to_sp_tensor(self, X):
        """Convert a scipy sparse matrix to tf.SparseTensor.
        Returns:
            tf.SparseTensor: SparseTensor after conversion.
        """
        coo = X.tocoo().astype(np.float32)
        indices = np.mat([coo.row, coo.col]).transpose()
        return tf.sparse.reorder(tf.SparseTensor(indices, coo.data, coo.shape))

    def cal_loss(self,u,p,n,test_mask,full=False):
        pos_score = tf.reduce_sum(tf.multiply(u, p), axis=-1) # (None, 1)
        neg_score = tf.reduce_sum(tf.multiply(u, n), axis=-1)
        
        
        if full:
            neg_score=tf.where(neg_score == 0,-1e10,neg_score)
        
        logits = tf.nn.softmax(tf.concat([pos_score, neg_score], axis=-1))
       
        #loss = tf.reduce_mean(- tf.math.log(tf.math.sigmoid(pos_score-neg_score)))
        loss=tf.reduce_mean(tf.math.softplus(neg_score-pos_score))
        
        return loss,logits
    
    

    def cosine_similarity(self,a, b):
        normalize_a = tf.math.l2_normalize(a,1)        
        normalize_b = tf.math.l2_normalize(b,1)
        cos_similarity=tf.linalg.matmul(normalize_a,normalize_b,transpose_b=True)
        return cos_similarity
    
    
    
    
    def cal_cl_loss(self,view1,view2,temp):
        cs=self.cosine_similarity(view1,view2)
        pos_cs=tf.linalg.diag_part(cs)
        loss=tf.math.reduce_mean(- tf.math.log(tf.math.exp(pos_cs/temp)/tf.math.reduce_sum(tf.math.exp(cs/temp),axis=1)))
        return loss
    

    def __init__(self,norm_adj,structure_fea):
        super(lightgcn, self).__init__()
        blocks=1
        embed_reg=0.
        layer_norm_eps=1e-6
        num_heads=1
        use_l2norm=False
        #self.u_project=tf.keras.layers.Dense(hidden_size)
        #self.i_project=tf.keras.layers.Dense(hidden_size)
        #self.vecs_np,self.p1,self.p2=vecs_np,p1,p2
        initializer_p = tf.keras.initializers.RandomNormal(mean=0., stddev=0.05,seed=2025)
        self.item_embedding_pool = tf.keras.layers.Embedding(items_num,hidden_size,input_length=items_num,
                                                       embeddings_initializer=initializer_p,
                                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg),trainable=True)
        self.user_embedding_pool = tf.keras.layers.Embedding(users_num,hidden_size,input_length=users_num,
                                                       embeddings_initializer=initializer_p,
                                        embeddings_regularizer=tf.keras.regularizers.l2(embed_reg),trainable=True)


        norm_adj=self._convert_sp_mat_to_sp_tensor(norm_adj)
        self.lg_layer=lightgcn_layer(norm_adj,structure_fea,hidden_size,0.25,1e-6)
        #self.a=tf.Variable(0.5)
        self.use_l2norm=use_l2norm
        self.temp=0.5
        
    def call(self,inputs,training=False,full=False):
        self.g1_ua_embeddings, self.g1_ia_embeddings=self.lg_layer(self.item_embedding_pool,self.user_embedding_pool,training)
   
        g1_u = tf.gather(
            params=self.g1_ua_embeddings, indices=tf.reshape(inputs['users'],[1,-1])
        )
        g1_p = tf.gather(
            params=self.g1_ia_embeddings, indices=tf.reshape(inputs['pos_item'],[1,-1])
        )
        
        
        indices = tf.tile(tf.reshape(tf.range(items_num), [1, items_num]),[tf.shape(inputs['pos_item'])[0],1])
        pos_idx_expanded = tf.tile(tf.reshape(inputs['pos_item'],[-1,1]), (1, items_num))
        test_mask = tf.math.not_equal(indices, pos_idx_expanded)#[b,1430]
        
        
        if full==True:
            g1_n=tf.expand_dims(self.g1_ia_embeddings,1)
        else:
            neg_indx=tf.random.uniform([tf.shape(inputs['pos_item'])[0],neg_num],dtype=tf.int32,maxval=items_num,minval=0,seed=2023)
            pos_idx = tf.repeat(tf.reshape(inputs['users'],[-1,1]), neg_num, axis=1)
            mask = tf.equal(neg_indx, pos_idx)
            neg_indx = tf.where(mask, tf.random.uniform([tf.shape(inputs['pos_item'])[0],neg_num],dtype=tf.int32,maxval=items_num,minval=0,seed=2024), neg_indx)   
            g1_n = tf.gather(
                params=self.g1_ia_embeddings, indices=tf.reshape(neg_indx,[tf.shape(neg_indx)[1],-1])
            )
        g1_u=tf.transpose(g1_u,[1,0,2])#[b,1,h]
        g1_p=tf.transpose(g1_p,[1,0,2])
        g1_n=tf.transpose(g1_n,[1,0,2])

        

        l1,logits1=self.cal_loss(g1_u,g1_p,g1_n,test_mask,full)
        
        bpr_loss=l1
        
        user_norm = tf.reduce_sum(tf.square(g1_u))
        pos_norm = tf.reduce_sum(tf.square(g1_p))
        neg_norm = tf.reduce_sum(tf.square(g1_n))
        reg_loss = 0.5 * (user_norm + pos_norm + neg_norm) / tf.cast(tf.shape(g1_n)[0], tf.float32)

        #reg_emb=tf.concat([self.item_embedding_pool(tf.range(items_num)),self.user_embedding_pool(tf.range(users_num))],axis=0)
        #reg_loss=0.00005*tf.math.reduce_mean(tf.norm(reg_emb))
        
        logits=logits1
        
        return logits,(bpr_loss,loss_ratio*reg_loss),(self.g1_ua_embeddings, self.g1_ia_embeddings)
    
    def test(self):
        g1_ua_embeddings, g1_ia_embeddings=self.lg_layer(self.item_embedding_pool,self.user_embedding_pool,training=False)
        score=tf.matmul(g1_ua_embeddings, g1_ia_embeddings,transpose_b=True)
        score=tf.where(score == 0,-1e10,score)
        return score
    def summary(self):
        inputs = {
            'users': tf.keras.layers.Input(shape=(), dtype=tf.int32),
            'pos_item': tf.keras.layers.Input(shape=(), dtype=tf.int32)
        }
        tf.keras.models.Model(inputs=inputs, outputs=self.call(inputs)).summary()

In [ ]:
import tqdm
import heapq
def hr_mrr_ndcf(rank,k):
    hr = 0.0
    mrr = 0.0
    ndcg = 0.0
    precision=0.0
    for r in rank:
        if r < k:
            if r<1:
                precision+=1
            hr += 1
            mrr += 1 / (r + 1)
            ndcg += 1 / np.log2(r + 2)
    return hr/len(rank),mrr/len(rank),ndcg/len(rank),precision/len(rank)

def eval_rank(pred_y, metric_names, k=10):
    """Evaluate
        Args:
            :param pred_y: A ndarray.
            :param metric_names: A list like ['hr'].
            :param k: A scalar(int).
        :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    rank = pred_y.argsort().argsort()[:, 0]
    res_dict = {}
    res=hr_mrr_ndcf(rank, k)
    for idx,name in enumerate(metric_names):
        res_dict[name] = res[idx]
    return res_dict

def eval_pos_neg(model, test_data, metric_names, k=10, batch_size=None):
    """Evaluate the performance of Top-k recommendation algorithm.
    Note: Test data must contain some negative samples(>= k) and one positive samples.
    Args:
        :param model: A model built-by tensorflow.
        :param test_data: A dict.
        :param metric_names: A list like ['hr'].
        :param k: A scalar(int).
        :param batch_size: A scalar(int).
    :return: A result dict such as {'hr':, 'ndcg':, ...}
    """
    whole_rating=model.test()
    whole_rating=whole_rating#.numpy()
    other_score_top_k=np.array([heapq.nlargest(k, row) for row in whole_rating.numpy()])
    #print(np.shape(other_score_top_k))
    print('start testing!')
    test_dataset = tf.data.Dataset.from_tensor_slices(test_data).batch(batch_size)
    final_dict={}
    for i in metric_names:
        final_dict.update({i:[]})
    for step, batch_test in tqdm.tqdm(enumerate(test_dataset)):
        cord=tf.concat([tf.reshape(batch_test['users'],[-1,1]),tf.reshape(batch_test['pos_item'],[-1,1])],axis=1)
        pos_score=tf.gather_nd(whole_rating,cord)
        pos_score=tf.reshape(pos_score,[-1,1])
        #pos_score =  model.test_pos_score(batch_test)
        #pos_score=tf.reshape(pos_score,[-1,1])
        #other_score=whole_rating[batch_test['users'].numpy()]
        other_score=tf.nn.embedding_lookup(whole_rating,batch_test['users'])
        #other_score_top_k=np.array([heapq.nlargest(k+1, row) for row in other_score])
        used_other_score_top_k=other_score_top_k[batch_test['users'].numpy()]
        pred_y=np.concatenate((pos_score,used_other_score_top_k),axis=1)
        #tf.print(pos_score.shape)
        #tf.print(other_score.shape)
        #pred_y=tf.nn.softmax(tf.concat([pos_score,other_score],axis=1))
        #pred_y=-pred_y.numpy()
        pred_y=-pred_y
        #print(pred_y[0])
        part_dict=eval_rank(pred_y, metric_names, k)
        for i in metric_names:
            final_dict [i].append(part_dict[i])
    for i in metric_names:
        final_dict [i]=np.mean(final_dict [i])
    return final_dict

In [ ]:
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]

In [ ]:
from time import time
from tensorflow.keras.backend import clear_session
import tensorflow as tf

number = 1
max_layers = 10
skip_epoch=10
ave_dict = {'hr': [], 'mrr': [], 'ndcg': [], 'precision': []}
tb = time()

train_dataset = tf.data.Dataset.from_tensor_slices(train_data).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices(val_data).batch(batch_size)

update_interval = int(len(train_dataset) / 1)
epoch_interval=10


@tf.function
def train_step_k(batch_train, model, optimizer_k):
    with tf.GradientTape() as tape1:
        logits, (bpr_loss,reg_loss), (g1_ua_embeddings, g1_ia_embeddings) = model(batch_train, training=True, full=False)
        loss_value = bpr_loss
    gradients1 = tape1.gradient(loss_value, [model.layers[2].trainable_weights[0]])
    optimizer_k.apply_gradients(zip(gradients1, [model.layers[2].trainable_weights[0]]))
    return loss_value, (bpr_loss,reg_loss), g1_ua_embeddings, g1_ia_embeddings

@tf.function
def train_step_query(batch_train, model, optimizer_query):
    with tf.GradientTape() as tape3:
        logits, (bpr_loss,reg_loss), (g1_ua_embeddings, g1_ia_embeddings) = model(batch_train, training=True, full=False)
        loss_value = bpr_loss
    gradients3 = tape3.gradient(loss_value, model.layers[2].trainable_weights)
    optimizer_query.apply_gradients(zip(gradients3, model.layers[2].trainable_weights))
    return loss_value, (bpr_loss,reg_loss), g1_ua_embeddings, g1_ia_embeddings

@tf.function
def train_step_emb(batch_train, model, optimizer_emb):
    with tf.GradientTape() as tape2:
        logits, (bpr_loss,reg_loss), (g1_ua_embeddings, g1_ia_embeddings) = model(batch_train, training=True, full=False)
        loss_value = bpr_loss
    gradients2 = tape2.gradient(loss_value, model.layers[0].trainable_weights + model.layers[1].trainable_weights)
    optimizer_emb.apply_gradients(zip(gradients2, model.layers[0].trainable_weights + model.layers[1].trainable_weights))
    
    return loss_value, (bpr_loss,reg_loss), g1_ua_embeddings, g1_ia_embeddings

@tf.function
def train_step_query_emb(batch_train, model, optimizer_query_emb):
    with tf.GradientTape() as tape2:
        logits, (bpr_loss,reg_loss), (g1_ua_embeddings, g1_ia_embeddings) = model(batch_train, training=True, full=False)
        loss_value = bpr_loss
    gradients2 = tape2.gradient(loss_value, model.layers[0].trainable_weights + model.layers[1].trainable_weights+model.layers[2].trainable_weights[1:])
    optimizer_query_emb.apply_gradients(zip(gradients2, model.layers[0].trainable_weights + model.layers[1].trainable_weights+model.layers[2].trainable_weights[1:]))
    
    return loss_value, (bpr_loss,reg_loss), g1_ua_embeddings, g1_ia_embeddings

@tf.function
def train_step(batch_train, model, optimizer_total):
    with tf.GradientTape() as tape:
        logits, (bpr_loss,reg_loss), (g1_ua_embeddings, g1_ia_embeddings) = model(batch_train, training=True, full=False)
        loss_value = bpr_loss+reg_loss
    gradients = tape.gradient(loss_value, model.trainable_weights)
    optimizer_total.apply_gradients(zip(gradients, model.trainable_weights))
    
    return loss_value, (bpr_loss,reg_loss), g1_ua_embeddings, g1_ia_embeddings

@tf.function
def val_step(model,batch_val):
    val_logits, val_loss,_ = model(batch_val, training=False, full=False)
    return val_loss[0]



model = lightgcn(norm_adj,structure_fea)
#model.summary()
for callback in callbacks:
    callback.set_model(model)
    callback.on_train_begin(logs={})

optimizer_k = tf.keras.optimizers.Nadam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
optimizer_query = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
optimizer_emb = tf.keras.optimizers.Nadam(learning_rate=0.0005, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
optimizer_total = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
optimizer_query_emb = tf.keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
model.compile()

for epoch in range(epoch_num):
    
    total_train_loss = 0.
    total_bpr_loss = 0.
    total_reg_loss = 0.
    total_cl_loss = 0.
    num_train_batches = 0.
    total_val_loss = 0.
    num_val_batches = 0.
    print(f"Epoch {epoch + 1}/{epoch_num}")
    t1 = time()
    if epoch>=skip_epoch:
        for callback in callbacks:
            callback.on_epoch_begin(epoch, logs={})

    for step, batch_train in enumerate(train_dataset):
        if step%100==0:
            print(step,'/',len(train_dataset))
        '''if epoch%5==0:
            loss_value,(bpr_loss,reg_loss),\
            g1_ua_embeddings,g1_ia_embeddings= train_step_k(batch_train,model,optimizer_k)
        else:
            loss_value,(bpr_loss,reg_loss),\
            g1_ua_embeddings,g1_ia_embeddings= train_step_k(batch_train,model,optimizer_k)
            loss_value,(bpr_loss,reg_loss),\
            g1_ua_embeddings,g1_ia_embeddings= train_step_query_emb(batch_train,model,optimizer_query_emb)'''
        loss_value,(bpr_loss,reg_loss),\
            g1_ua_embeddings,g1_ia_embeddings= train_step(batch_train,model,optimizer_total)
        #loss_value,bpr_loss,\
        #    g1_ua_embeddings,g1_ia_embeddings= train_step_k(batch_train,model,optimizer_k)
        #loss_value,bpr_loss,\
        #    g1_ua_embeddings,g1_ia_embeddings= train_step_query(batch_train,model,optimizer_query)
        #loss_value,bpr_loss,\
        #    g1_ua_embeddings,g1_ia_embeddings= train_step_emb(batch_train,model,optimizer_emb)
        total_train_loss += loss_value
        total_bpr_loss += bpr_loss
        total_reg_loss += reg_loss
        num_train_batches += 1

    avg_train_loss = total_train_loss / num_train_batches
    avg_bpr_loss = total_bpr_loss / num_train_batches
    avg_reg_loss = total_reg_loss / num_train_batches
    
    print(f"Time cost: {time()-t1:.2f}, Training loss: {avg_train_loss.numpy():.4f}, bpr loss: {avg_bpr_loss.numpy():.4f}, reg loss: {avg_reg_loss.numpy():.4f}")
    t2=time()
    for batch_val in val_dataset:
        val_loss = val_step(model,batch_val)
        #val_loss = val_step(model,batch_val,ori_A_norm,ori_L_norm)
        total_val_loss += val_loss
        num_val_batches += 1

    avg_val_loss = total_val_loss / num_val_batches
    print(f"Time cost: {time()-t2:.2f}, Validation loss: {avg_val_loss.numpy():.4f}")
    
    if epoch>=skip_epoch:
        for callback in callbacks:
            callback.on_epoch_end(epoch, logs={'val_loss': avg_val_loss})
    if any([callback.stopped_epoch for callback in callbacks]):
        break

for callback in callbacks:
    callback.on_train_end()

In [ ]:
tt = time()
eval_dict = eval_pos_neg(model, test_data,['hr', 'mrr', 'ndcg', 'precision'], 10, batch_size)
#eval_dict = eval_pos_neg(model, test_data,ori_A_norm,ori_L_norm, ['hr', 'mrr', 'ndcg', 'precision'], 10, batch_size * 10)
for m in ['hr', 'mrr', 'ndcg', 'precision']:
    ave_dict[m].append(eval_dict[m])
print('Precision = %.4f, HR = %.4f, MRR = %.4f, NDCG = %.4f'
      % (eval_dict['precision'], eval_dict['hr'], eval_dict['mrr'], eval_dict['ndcg']))